In [1]:
-- 1.
SELECT c.cid, c.name from Customer c
LEFT JOIN Ordered o ON c.cid = o.cid
LEFT JOIN Tour t ON o.tid = t.tid
LEFT JOIN TourGuide tg ON t.tgid = tg.tgid
WHERE c.frequentTravelerStatus = 'silver' AND tg.name = 'B. Grylls'

(1 row affected)

Total execution time: 00:00:00.027

cid,name
2,Sabina


In [2]:
-- 2.
SELECT MAX(c.age) AS oldest_age
FROM customer c
JOIN ordered o ON c.cid = o.cid
JOIN tour t ON o.tid = t.tid
JOIN tourGuide tg ON t.tgid = tg.tgid
WHERE c.countryOfResidence = 'Germany' OR tg.name = 'B. Grylls'

(1 row affected)

Total execution time: 00:00:00.023

oldest_age
66


In [3]:
-- 3.
SELECT DISTINCT t.destinationCountry AS tourName
FROM tour t
WHERE t.destinationCountry = 'Oman' OR 
t.tid IN (SELECT o.tid FROM Ordered o GROUP BY o.tid HAVING COUNT(DISTINCT o.cid) >= 5)

(2 rows affected)

Total execution time: 00:00:00.023

tourName
Bosnia and Herzegovina
Oman


In [4]:
-- 4.
SELECT c.cid, c.name FROM Customer c
JOIN Ordered o ON c.cid = o.cid
JOIN Tour t ON o.tid = t.tid
GROUP BY t.tourDate, c.cid, c.name
HAVING (COUNT(t.tid)>=2)

(3 rows affected)

Total execution time: 00:00:00.022

cid,name
1,Linda
2,Sabina
3,John


In [5]:
-- 5.
SELECT tg.tgid, tg.name FROM TourGuide tg
JOIN tour t ON tg.tgid = t.tgid
GROUP BY tg.tgid, tg.name
HAVING COUNT(DISTINCT t.destinationCountry) = (SELECT COUNT(DISTINCT t.destinationCountry) FROM Tour t)
           

(1 row affected)

Total execution time: 00:00:00.021

tgid,name
1,Kamber


In [6]:
-- 6.
SELECT TG.name, TG.tgid FROM TourGuide TG
JOIN Tour t ON TG.tgid = t.tgid
GROUP BY TG.tgid, TG.name
HAVING COUNT(t.tid) < 3

(3 rows affected)

Total execution time: 00:00:00.020

name,tgid
Manuel,2
B. Grylls,3
RandomGuy,5


In [7]:
-- 7.
SELECT AVG(age) as averageAge, frequentTravelerStatus
FROM Customer
WHERE age > 40
GROUP BY  frequentTravelerStatus
HAVING AVG(age) < 50

(1 row affected)

Total execution time: 00:00:00.022

averageAge,frequentTravelerStatus
46,silver


In [8]:
-- 8.
SELECT tg.tgid, tg.name, COUNT(t.tid) AS total_tours
FROM tourGuide tg
JOIN tour t ON tg.tgid = t.tgid
GROUP BY tg.tgid, tg.name
HAVING COUNT(DISTINCT t.destinationCountry) = 1 AND MAX(t.destinationCountry) = 'Oman'

(1 row affected)

Total execution time: 00:00:00.024

tgid,name,total_tours
5,RandomGuy,2


In [9]:
-- 9.
SELECT c.cid, c.name
FROM customer c
JOIN ordered o ON c.cid = o.cid
GROUP BY c.cid, c.name
HAVING COUNT(o.tid) = (SELECT COUNT(DISTINCT t.tid) FROM tour t)

(0 rows affected)

Total execution time: 00:00:00.027

cid,name


In [10]:
-- 10.
SELECT * FROM Customer c WHERE c.cid NOT IN (SELECT DISTINCT(o.cid) FROM Ordered o)

(1 row affected)

Total execution time: 00:00:00.024

cid,name,countryOfResidence,age,frequentTravelerStatus
7,Peter,Germany,70,diamond


In [11]:
-- 11.
WITH StatusCounts AS (
    SELECT c.age, c.frequentTravelerStatus, COUNT(*) AS status_count,
            ROW_NUMBER() OVER (PARTITION BY c.age ORDER BY COUNT(*) DESC) AS sequence_number
        FROM customer c
        GROUP BY c.age, c.frequentTravelerStatus)
SELECT age, frequentTravelerStatus, status_count
    FROM StatusCounts
    WHERE sequence_number = 1

(5 rows affected)

Total execution time: 00:00:00.027

age,frequentTravelerStatus,status_count
26,diamond,2
45,diamond,1
46,silver,1
66,platinum,1
70,diamond,1
